In [1]:
# packages
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import time

In [2]:
covid_cases = pd.read_csv('data_table_for_daily_case_trends__the_united_states(1).csv', skiprows = 2)
covid_deaths = pd.read_csv('data_table_for_daily_death_trends__the_united_states.csv', skiprows = 2)
vaxx = pd.read_csv('trends_in_number_of_covid19_vaccinations_in_the_us.csv', skiprows = 2)
cpi = pd.read_excel('SeriesReport-20220408081955_015b2d.xlsx', skiprows = 11)
unemp = pd.read_excel('SeriesReport-20220408081142_3fb54b.xlsx', skiprows = 11)

# Melt / reformat cpi & unemployment
cpi = cpi.melt(id_vars = ['Year'],
        var_name = 'Month', 
        value_name = 'CPI')
unemp = unemp.melt(id_vars = ['Year'], 
           var_name = 'Month', 
           value_name = 'Unemployment Rate')
# Can only run once
cpi = cpi[~cpi['Month'].str.contains('HALF')]
cpi['Date'] = pd.to_datetime(cpi['Month'] + '-' + cpi['Year'].astype(str))
unemp['Date'] = pd.to_datetime(unemp['Month'] + '-' + unemp['Year'].astype(str))
unemp = unemp.sort_values(by = ['Date'], ascending = True)
cpi = cpi.sort_values(by = ['Date'], ascending = True)
# Only select dates after 2020
unemp = unemp[unemp['Date'].dt.year > 2019]
# cpi = cpi[cpi['Date'].dt.year > 2019]
# Clean Covid / Vaxx frames
covid_cases['Date'] = pd.to_datetime(covid_cases['Date'])
covid_deaths['Date'] = pd.to_datetime(covid_deaths['Date'])
vaxx['Date'] = pd.to_datetime(vaxx['Date'])
vaxx['Month'] = pd.DatetimeIndex(vaxx['Date']).month
covid_cases['Month'] = pd.DatetimeIndex(covid_cases['Date']).month
covid_deaths['Month'] = pd.DatetimeIndex(covid_deaths['Date']).month
vaxx['Year'] = pd.DatetimeIndex(vaxx['Date']).year
covid_cases['Year'] = pd.DatetimeIndex(covid_cases['Date']).year
covid_deaths['Year'] = pd.DatetimeIndex(covid_deaths['Date']).year
# Roll vaxx/covid up to monthly level -- add date columns back in
# covid_cases = covid_cases.groupby(by = ['State', 'Year','Month']).sum().reset_index()
# covid_deaths = covid_deaths.groupby(by = ['State', 'Year','Month']).sum().reset_index()
# vaxx = vaxx.groupby(by = ['Long Name', 'Year','Month']).sum().reset_index()
# vaxx['Date'] = pd.to_datetime(vaxx['Month'].astype(str) + '-' + vaxx['Year'].astype(str))
# covid_cases['Date'] = pd.to_datetime(covid_cases['Month'].astype(str) + '-' + covid_cases['Year'].astype(str))
# covid_deaths['Date'] = pd.to_datetime(covid_deaths['Month'].astype(str) + '-' + covid_deaths['Year'].astype(str))

In [9]:
covid_deaths

# normalized_df=(df-df.min())/(df.max()-df.min())
covid_deaths['Normalized Deaths'] = (covid_deaths['New Deaths'] - covid_deaths['New Deaths'].min()) / (covid_deaths['New Deaths'].max() - covid_deaths['New Deaths'].min())
covid_cases['Normalized Cases'] = (covid_cases['New Cases'] - covid_cases['New Cases'].min()) / (covid_cases['New Cases'].max() - covid_cases['New Cases'].min())
covid_cases

covid_cases = covid_cases.groupby(['State']).resample('W-Wed', label='right', closed = 'right', on='Date').sum().reset_index().sort_values(by='Date')
covid_deaths = covid_deaths.groupby(['State']).resample('W-Wed', label='right', closed = 'right', on='Date').sum().reset_index().sort_values(by='Date')
vaxx = vaxx.groupby(['Long Name']).resample('W-Wed', label='right', closed = 'right', on='Date').sum().reset_index().sort_values(by='Date')

In [73]:
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot covid cases
fig.add_trace(go.Bar(
    x = covid_cases['Date'], 
    y = covid_cases['Normalized Cases'], 
    name = 'New Covid Cases',
    opacity = 1,
    marker_color = 'blue'
    # fill = 'tozeroy' # only for Scatter
))
# Plot covid deaths
fig.add_trace(go.Bar(
    x = covid_deaths['Date'], 
    y = covid_deaths['Normalized Deaths'],
    name = 'New Covid Deaths',
    opacity = 0.7,
    marker_color = 'red'
    # fill = 'tozeroy'
))
# Plot Vaxxinations administered
vaxx.loc[vaxx['Date'] == vaxx['Date'].max(), 'Administered'] = np.nan # rm latest month of data due to incompleteness
fig.add_trace(go.Scatter(
    x = vaxx['Date'], 
    y = vaxx['Administered'], 
    marker = dict(symbol = 'square'),
    name = 'Vaccinations Administered',
),secondary_y = True)
# Plot Unemployment rate
# fig.add_trace(go.Scatter(
#     x = unemp['Date'], 
#     y = unemp['Unemployment Rate'], 
#     name = 'Unemployment Rate', 
#     yaxis = 'y2',
#     line = dict(dash = 'dash')
# ))
# Plot Inflation
# fig.add_trace(go.Scatter(
#     x = cpi['Date'], 
#     y = cpi['Inflation'], 
#     name = 'Inflation', 
#     yaxis = 'y4'
# ))


# Create axis objects
fig.update_layout(title = {
                  'text':'COVID-19 Cases, Deaths, & Vaccinations', 
                  'x':0.4,
                  'xanchor':'center'
                  },
                  xaxis=dict(title = 'Time'),
                  yaxis=dict(title = 'Normalized Covid-19<br>Cases & Deaths',
                  side = 'left', 
                  position = 0.0000000000000000000000000025), 

                  yaxis2=dict(title = 'Vaccinations',
                  anchor = 'x', 
                  overlaying = 'y', 
                  side = 'right'),

                  yaxis3 = dict(title = 'Unemployment Rate<br>(%)',
                  anchor = 'x',
                  overlaying = 'y',
                  side = 'right'),
                  height = 450, 
                  width = 1100)

# Omicron BA.1
fig.add_annotation(x='2021-9-15', y=0.65,
            text="Omicron BA.1",
            showarrow=True,
            arrowhead=1)

fig.add_shape(type="rect",
    xref="x", yref="y",
    x0="2021-8-1", y0=0.6, x1="2021-10-28", y1=0,
    line=dict(
        color="orange",
        width=3,
    ),
)
# Omicron BA.2
fig.add_annotation(x='2022-1-28', y=1.04,
            text="Omicron BA.2",
            showarrow=True,
            arrowhead=1)

fig.add_shape(type="rect",
    xref="x", yref="y",
    x0="2021-12-20", y0=1.02, x1="2022-2-28", y1=0,
    # opacity = 0.8,
    line=dict(
        color="yellow",
        width=3,
    ),
)
# Delta Identifier
fig.add_annotation(x='2021-1-3', y=1.05,
            text="Delta Varient",
            showarrow=False,
            arrowhead=1)

fig.add_shape(type="rect",
    xref="x", yref="y",
    x0="2020-11-15", y0=1, x1="2021-2-20", y1=0,
    # opacity = 0.8,
    line=dict(
        color="yellow",
        width=3,
    ),
)

note_subheader = "Notes: Due to the difference in scale between cases (peaks at 20M/daily for Omicron) and deaths (peaks at 5K/daily for Delta)<br>I have forced them down to the same scale to better comapre the trend between the two"
note_footer = "Source: https://covid.cdc.gov/covid-data-tracker/<br>Graphic Created by: @Longboi_Mat"
fig.add_annotation(
    showarrow=False,
    text=note_subheader,
    font=dict(size=10), 
    xref='x domain',
    x=0.5,
    yref='y domain',
    y=1.15
    )
fig.add_annotation(
    showarrow=False,
    text=note_footer,
    font=dict(size=10), 
    xref='x domain',
    x=0.5,
    yref='y domain',
    y=-0.29
    )
# fig.update_yaxes(rangemode='tozero', scaleanchor='y2', scaleratio=1, constraintoward='bottom', secondary_y=True)
# fig.update_yaxes(rangemode = 'tozero', scaleanchor='y1', scaleratio=1, constraintoward='bottom', secondary_y = False)
fig['layout']['yaxis2']['showgrid'] = False
fig.show()
fig.write_html("covid file.html")
covid_deaths.head()

,State,Date,New Deaths,7-Day Moving Avg,Historic Deaths,Month,Year,Normalized Deaths
0,United States,2020-01-29,0,0,0,7,14140,0.0
1,United States,2020-02-05,0,0,0,12,14140,0.0
2,United States,2020-02-12,0,0,0,14,14140,0.0
3,United States,2020-02-19,0,0,0,14,14140,0.0
4,United States,2020-02-26,0,0,0,14,14140,0.0


In [5]:
px.line(unemp, x = 'Date', y = 'Unemployment Rate')
print(unemp['Date'].min())
print(covid['Date'].min())
print(vaxx['Date'].min())

2020-01-01 00:00:00


NameError: name 'covid' is not defined

In [ ]:
px.line(cpi, x = 'Date', y = 'CPI')